-Ban (champions) en % 
-Overall
-Red Side
-Pour et contre l'équipe

ECONOMY :
-Gold/Minute
-ΔGold/Minute
-ΔGold@15
-Win Rate when ahead at 15 min
-CS/Minute
-ΔCS@15
-ΔTower@15
-Average ΔTower
-First Tower

AGGRESSION :
-Damage/Minute
-First Blood %
-Kill/Game
-Death/Game
-Damage/Gold
-Solo Kill
-First Blood Victime
-Death@10

PLAYER STATS :
-Role        
-Player 
-name    
-KDA    
-KP%    
-VS/M    
-DMG%    
-GOLD%    
-CS/M    
-DMG/GOLD    
-ΔGold@15

OBJECTIVES :
-Drake / Game
-ΔDrake@15
-First Drake%
-Drake Own        
-Drake Lose       
-Drake Own%
-Herald / Game
-First Herald%
-Herald Own        
-Herald Lose        
-Herald Own %
-First Nashor%
-Nashor Own        
-Nashor Lose        
-Nashor Own %

VISION : 
-Vision Score Per Minute  
-Wards Per Minute    
-Vision Wards Per Minute
-Wards Cleared Per Minute
-% Wards Cleared

EARLY GAME  (Joueur) :
-Ahead in CS @15
-ΔCS@15
-ΔGold@15
-First Blood Participation
-First Blood Victime

VISION (Joueur):
-VisionScore Per Minute
-VisionScore Per Game
-Ward Per Minute
-Ward Per Game
-Ward Cleared Per Minute
-Ward Cleared Per Game

AGRESSION(Joueur):
-DMG Per Minute
-DMG %
-Solo Kills        
-Triple Kills        
-Quadra Kills        
-Penta Kills        
-Death@10

In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
from champ import donnee
from item import items
from summ import datasumm

In [2]:
lol_watcher = LolWatcher('RGAPI-2094315c-31a1-4d3d-953f-e4441756364a')
my_region = 'EUW1'

In [3]:
me = lol_watcher.summoner.by_name(my_region, 'KC Isnow')
print(me)

{'id': 'Z2bfMio01qPjPdWaMX_2Xr89PT2IDmi4iuHFbS2eazTGQl5Tdbe4aFLWcA', 'accountId': '1Q69y9VDie8nz__-dBgm_IlQ0McrVp3rrFI8U1_Lrdbmzi0rHNOBTmxK', 'puuid': 'ROukO09gYPGXJpWcKB8MTx9si7e7tzTJDLsr9cdg-u94-n48LXX19FjyQ92ZsLPkYjFYivqxbh6McQ', 'name': 'KC Isnow', 'profileIconId': 4882, 'revisionDate': 1622662117000, 'summonerLevel': 37}


In [4]:
def findName(donnee,key):
    for perso in donnee["data"]:
        if donnee["data"][perso]["key"] == str(key):
            return donnee["data"][perso]["name"]
    return False


In [5]:
def findSumm(datasumm, key):
    for summoners in datasumm["data"]:
        if datasumm["data"][summoners]["key"] == str(key):
            return datasumm["data"][summoners]["name"]
    return False

In [6]:
def findItem (items, key):
    for objet in items["data"]:
        if objet == str(key):
            return items["data"][objet]["name"]
    return False

In [7]:
def getCSDiffTen(match,index):
    if 'csDiffPerMinDeltas' in match['participants'][index]['timeline']:
        return match['participants'][index]['timeline']['csDiffPerMinDeltas']['0-10']
    else:
        return False

In [26]:
"""
Exemple : 
[{'championId': 32, 'pickTurn': 1}, {'championId': 34, 'pickTurn': 3}, {'championId': 432, 'pickTurn': 5}, {'championId': 84, 'pickTurn': 2}, {'championId': 1, 'pickTurn': 4}]
"""
def findAllBans(listeBan):
    allBan = []
    for ban in listeBan:
        allBan.append(findName(donnee,ban['championId']))
    return allBan

In [15]:
listmatch = [5265594334, 5265595024 , 5265595794 , 5260412449, 5260414236, 5260867926 , 5261356635 , 5261356635 , 5263645425 , 5263646324 , 5263297168 ]
i = 0
dg = []
writer = pd.ExcelWriter(r'C:\Data riot\test1.xlsx')
for code in listmatch:
    matchesdetail = lol_watcher.match.by_id(my_region, code)
    
    participants = []

    for row in matchesdetail['participants']:
        participants_row = {}
        participants_row['champion'] = findName(donnee,row["championId"])
        participants_row['lane'] = row['timeline']['lane']
        participants_row['role'] = row['timeline']['role']
        participants_row['champLevel'] = row['stats']['champLevel']
        participants_row['spell1'] = findSumm(datasumm,row['spell1Id'])
        participants_row['spell2'] = findSumm(datasumm,row['spell2Id'])
        participants_row['win'] = row['stats']['win']
        participants_row['kills'] = row['stats']['kills']
        participants_row['deaths'] = row['stats']['deaths']
        participants_row['assists'] = row['stats']['assists']
        participants_row['cs'] = row['stats']['totalMinionsKilled']
        participants_row['creep per min @10'] = row['timeline']['creepsPerMinDeltas']['0-10']
        participants_row['cs diff @10'] = getCSDiffTen(matchesdetail,row["participantId"]-1)
        participants_row['totalDamageDealtToChampions'] = row['stats']['totalDamageDealtToChampions']
        participants_row['goldEarned'] = row['stats']['goldEarned']
        participants_row['champLevel'] = row['stats']['champLevel']
        participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
        participants_row['doubleKills'] = row['stats']['doubleKills']
        participants_row['tripleKills'] = row['stats']['tripleKills']
        participants_row['quadraKills'] = row['stats']['quadraKills']
        participants_row['pentaKills'] = row['stats']['pentaKills']
        participants_row['fristblood kill'] = row['stats']['firstBloodKill']
        participants_row['fristblood kill assist'] = row['stats']['firstBloodAssist']
        participants_row['frist tower kill'] = row['stats']['firstTowerKill']
        participants_row['frist tower assist'] = row['stats']['firstTowerAssist']
        participants_row['visions'] = row['stats']['visionScore']
        participants_row['pinkward'] = row['stats']['visionWardsBoughtInGame']
        participants_row['ward pose'] = row['stats']['wardsPlaced']
        participants_row['ward eneleve'] = row['stats']['wardsKilled']
        participants_row['item0'] = findItem (items, row['stats']['item0'])
        participants_row['item1'] = findItem (items, row['stats']['item1'])
        participants_row['item2'] = findItem (items, row['stats']['item2'])
        participants_row['item3'] = findItem (items, row['stats']['item3'])
        participants_row['item4'] = findItem (items, row['stats']['item4'])
        participants_row['item5'] = findItem (items, row['stats']['item5'])
        
        participants.append(participants_row)
    df = pd.DataFrame(participants)
    dg.append(df)

for b in dg :
    i += 1 
    b.to_excel(writer, sheet_name = str(i))


writer.save()

In [27]:
i = 0
dg = []
writer = pd.ExcelWriter(r'C:\Data riot\statteam.xlsx')
for code in listmatch:
    matchesdetail = lol_watcher.match.by_id(my_region, code)
    
    teams = []

    for row in matchesdetail['teams']:
        teams_row = {}
        teams_row['side'] = ('blueside' if row['teamId'] == 100 else 'redside') 
        teams_row['win'] = row['win']
        teams_row['firstBlood'] = row['firstBlood']
        teams_row['firstBaron'] = row['firstBaron']
        teams_row['firstDragon'] = row['firstDragon']
        teams_row['firstRiftHerald'] = row['firstRiftHerald']
        teams_row['towerKills'] = row['towerKills']
        teams_row['inhibitorKills'] = row['inhibitorKills']
        teams_row['baronKills'] = row['baronKills']
        teams_row['riftHeraldKills'] = row['riftHeraldKills']
        teams_row['bans'] = findAllBans(row['bans'])
        teams_row['riftHeraldKills'] = row['inhibitorKills']

        
        teams.append(teams_row)
    df = pd.DataFrame(teams)
    dg.append(df)
    
for b in dg :
    i += 1 
    b.to_excel(writer, sheet_name = str(i))


writer.save()